##### Imports

In [1]:
import asvFormula.experiments as exp
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula import asvRunResultsPath
from asvFormula.datasetManipulation import *
from asvFormula.digraph import hasUnderlyingTree
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination

from importlib import reload 
import asvFormula
import asvFormula.experiments as exp

exp = reload(asvFormula.experiments)

#TODO: Fix the seed and the random so that I can reproduce the results. Right now it's not changing anything the seed method. 

### Running ASV for Cancer bayesian network

#### Train the decision tree model

In [2]:
cancerNetworkPath = networkSamplesPath + "/cancer.bif"
BNmodel = BIFReader(cancerNetworkPath).get_model()
variableToPredict = "Pollution"
numberOfSamples = 2000
treeMaxDepth = 5

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(BNmodel, variableToPredict, numberOfSamples, treeMaxDepth)
#drawDecisionTree(dtAsNetwork)

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.905


#### Mean prediction of model

In [3]:
dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

asvCalc = exp.ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, 'Exact', first_instance)

# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(cancerNetworkPath).get_model()
num_variables = len(completeBNModel.nodes())

completeBNInference = VariableElimination(completeBNModel)

exp.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier, asvCalc, num_variables)


Mean prediction of model for the variable Pollution
Mean prediction value for the decision tree: [0.0909602 0.9090398], it took 0.009067400000276393 seconds
Mean prediction value for possible values of the dataset: [0.007350000000000001, 0.99265], it took 0.01649730000008276 seconds
Estimated value for shap explainer: [0.09125 0.90875]
Probabilities of the variable in the bayesian network: [0.1, 0.9]


In [4]:
variableToPredict

'Pollution'

### Running ASV for Child bayesian network

#### Train the decision tree model

In [5]:
childNetworkPath = networkSamplesPath + "/child.bif"

def childBNAsTree(childNetworkPath):
    treeBNChild = BIFReader(childNetworkPath).get_model()

    #I remove this edges so that it is a tree and we can work with it
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'Grunting')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungParench', 'HypoxiaInO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'HypoxiaInO2', 'LowerBodyO2')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'CardiacMixing', 'HypDistrib')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'Sick', 'Age')
    removeEdgeAndMarginalizeCPD(treeBNChild, 'LungFlow', 'ChestXray')

    assert hasUnderlyingTree(treeBNChild)
    return treeBNChild

treeBNChild = childBNAsTree(childNetworkPath)
variableToPredict = "Age"
numberOfSamples = 10000
treeMaxDepth = 7

BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth)


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.6755


#### Mean prediction of model

In [9]:
dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

asvCalc = exp.ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, 'Exact', first_instance)

# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(childNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)
num_variables = len(completeBNModel.nodes())

exp.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier, asvCalc, 11)

#The prediction for the first instance is 0, so if a lot of the features are fixed, then there is a higher chance that the prediction is 0.


Mean prediction of model for the variable Age
Mean prediction value for the decision tree: [0.68851015 0.13590638 0.17604583], it took 0.1552386090006621 seconds
Mean prediction value for possible values of the dataset: [0.996717443108697, 0.000578492552145865, 0.002704064339157585], it took 116.19580318800035 seconds
Estimated value for shap explainer: [0.691125 0.13525  0.173625]
Probabilities of the variable in the bayesian network: [0.6489918355500001, 0.17148510286900004, 0.17952306158100004]


#### ASV + Shapley

In [ ]:
asvCalc = exp.ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction = 'Mean')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

exp.writeASVAndShapleyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/childASVAndShapleyMean.csv", valuesPerFeature, variableToPredict)

In [ ]:
asvCalc = exp.ASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Exact')

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

exp.writeASVAndShapleeyIntoFile(first_instance, list(dataNoPrediction.columns), dtTreeClassifier, asvCalc, asvRunResultsPath + "/childASVAndShapleyExact.csv", valuesPerFeature, variableToPredict)

#### Mean Prediction

In [ ]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(childNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [ ]:
exp.showMeanPredictionOfModel(variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

In [ ]:
dtAsNetwork = exp.obtainDecisionTreeDigraph(dtTreeClassifier, dataNoPrediction.columns)
#exp.drawDecisionTree(dtAsNetwork)
#exp.drawGraph(treeBNChild)